# Look at our Evaluations

In [1]:
%load_ext autoreload

# Automatically reload all modules (except those excluded by %aimport)
%autoreload 2

In [2]:
import sys
import utils

In [3]:
data = utils.read_dataset('FB15K-237-N')

2024-03-10 08:43:53,227 utils INFO: Reading dataset FB15K-237-N...
Saved to f'pos_tmp.jsonl
Saved to f'neg_tmp.jsonl


In [4]:
save_path = '../data/results/CoDeX-S/WebKGValidator/150_WebKGValidator_CoDeX-S_seed9_gpt-3.5-turbo-0125.jsonl'


In [5]:
results = utils.read_jsonl(save_path)
len(results)

1

In [11]:
results[0]

{'negative_results': {'triples': [], 'validated_triples': []},
 'positive_results': {'triples': [], 'validated_triples': []}}

In [6]:
tp = 0
fp = 0
tn = 0
fn = 0 
pos_results = results[0]['positive_results']
neg_results = results[0]['negative_results']
for val in pos_results['validated_triples']:
    if val['triple_is_valid']:    # property is correctly marked as valid
        tp += 1
    else:                           # property is incorrectly marked as invalid
        fn += 1
for val in neg_results['validated_triples']:
    if val['triple_is_valid']:    # property is incorrectly marked as valid
        fp += 1
    else:                           # property is correctly marked as invalid
        tn += 1

metrics = utils.calc_metrics(tp, fp, tn, fn)

# Precision: 0.75
# Recall: 0.8571428571428571
# F1 Score: 0.7999999999999999
# Accuracy: 0.7857142857142857
# ----------

Precision: 0
Recall: 0
F1 Score: 0
Accuracy: 0
----------


In [7]:
# what did the evaluator get wrong?
import json
false_neg = 0
false_pos = 0
unsure = 0
print('\n----------------False Negatives------------------\n')
for val in pos_results['validated_triples']:
    if val['triple_is_valid'] is False:
        print(json.dumps(val, indent=2))
        false_neg += 1
print(f"Total False Negatives: {false_neg}")


print('\n----------------False Positives------------------\n')
for val in neg_results['validated_triples']:
    if val['triple_is_valid'] is True:
        print(json.dumps(val, indent=2))
        false_pos += 1
print(f"Total False Positives: {false_pos}")


print('\n----------------Unsure (Not enough info to say)------------------\n')
for val in neg_results['validated_triples']:
    if val['triple_is_valid'] not in [True, False]:
        print(json.dumps(val, indent=2))
        unsure += 1
print(f"Total Unsure: {unsure}")




----------------False Negatives------------------

Total False Negatives: 0

----------------False Positives------------------

Total False Positives: 0

----------------Unsure (Not enough info to say)------------------

Total Unsure: 0


In [8]:
print(f"Total Unsure: {unsure}")
print(f"Total False Negatives: {false_neg}")
print(f"Total False Positives: {false_pos}")

Total Unsure: 0
Total False Negatives: 0
Total False Positives: 0


In [9]:
for val in neg_results['validated_triples']:
    if val['triple_is_valid'] is False:
        print(json.dumps(val, indent=2))

# Embeddings 🛌

In [1]:
from embedding_models import EmbeddingModelFactory
import os
os.environ['VALIDATION_MODEL'] = 'gpt-3.5-turbo-0125'
os.environ['EMBEDDING_MODEL'] = 'all-MiniLM-L6-v2'
EMBEDDING_MODEL = EmbeddingModelFactory.get_embedding_model(os.environ['EMBEDDING_MODEL'])  #  'mixedbread-ai/mxbai-embed-large-v1'  /  'all-MiniLM-L6-v2'
if hasattr(EMBEDDING_MODEL, 'name'):
    # EMBEDDING_MODEL.load_model()
    print(EMBEDDING_MODEL.name)
else:
    print(EMBEDDING_MODEL.model)

all-MiniLM-L6-v2


In [3]:
import duckduckgo_verbose_search
import json
from langchain_core.documents.base import Document
from embedding_models import EmbeddingModelFactory
import os

import validator_utils
import validators

subject, relation, object = "Roy Marlin Voris", "nationality", "Tahiti"
query = f"What {subject} {relation} {object}?"

web_searcher = duckduckgo_verbose_search.DuckDuckGoVerboseSearch(max_search_results=5)
web_results = web_searcher(query)
# print(json.dumps(web_results, indent=2))
web_reference = [Document(f"{result['title']} {result['body']}") for result in web_results]
retriever, store, vectorstore = validator_utils.create_parent_document_retriever(web_reference, embedding_function=EMBEDDING_MODEL)

relevant_chunks = validator_utils.retrieve_relevant_chunks(
    query=validators.WebKGValidator.create_query(subject, relation, object), 
    vectorstore=vectorstore,
    retriever=retriever,
)
relevant_chunks

2024-03-11 12:07:34,252 validators INFO: Using Validator model gpt-3.5-turbo-0125
2024-03-11 12:07:36,533 validators INFO: Using embedding model all-MiniLM-L6-v2


['Roy Marlin Voris - Wikipedia \n\nRoy Marlin Voris - Wikipedia\n\nJump to content\n\nMain menu\n\nMain menu\nmove to sidebar\nhide\n\n Navigation\n\n* Main page\n* Contents\n* Current events\n* Random article\n* About Wikipedia\n* Contact us\n* Donate\n\n Contribute\n\n* Help\n* Learn to edit\n* Community portal\n* Recent changes\n* Upload file\n\n![]()\n\n![Wikipedia]()\n![The Free Encyclopedia]()\n\nSearch\n\nSearch\n\n* Create account\n* Log in\n\nPersonal tools\n\n* Create account\n* Log in\n\n Pages for logged out editors learn more',
 '|  |\n| --- |\n| **First Name:** Roy\n**Last Name:** Voris\n**Birthplace:** Los Angeles, CA, USA\n**Gender:** Male\n**Branch:** Navy (present)\n**Rating:**  |\n\n | \n\n|  |\n| --- |\n| **Middle Name:** Marlin \n\n**Date of Birth:** 19 September 1919\n**Date of Death:** 10 August 2005 \n**Rank or Rate:** Captain \n**Years Served:** |\n\n |\n| \n Roy Marlin Voris \n \'Butch\' |  |  |\n| **Engagements:** •\xa0\xa0World War II (1941 - 1945) |\n| **Bi